In [2]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO
from google.datalab import Context
import random, string
import numpy as np
import matplotlib.pyplot as plt

In [3]:
shared_bucket = storage.Bucket('line_capstone')
for obj in shared_bucket.objects():
  if obj.key.find('/') < 0:
    print(obj.key) # Bucket 내에 포함된 데이터 셋 확인 

clicks_test.csv
clicks_train.csv
documents_categories.csv
documents_entities.csv
documents_meta.csv
documents_topics.csv
events.csv
page_views_sample.csv
promoted_content.csv
sample_submission.csv


In [17]:
my_bucket = storage.Bucket('line_capstone')

docu_meta = my_bucket.object('documents_meta.csv')
uri5 = docu_meta.uri
%gcs read --object $uri5 --variable c_docu_meta
docu_meta = pd.read_csv(BytesIO(c_docu_meta))

pro_cont = my_bucket.object('promoted_content.csv')
uri9 = pro_cont.uri
%gcs read --object $uri9 --variable c_pro_cont
promoted = pd.read_csv(BytesIO(c_pro_cont))

events = my_bucket.object('events.csv')
uri7 = events.uri
%gcs read --object $uri7 --variable c_events
events = pd.read_csv(BytesIO(c_events))

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
# promoted와 docu_meta를 document_id를 key로 해서 merge
merged = pd.merge(events, docu_meta, how='inner', on='document_id')

In [24]:
# docu_meta에 NaN 값이 이미 많다.
pd.isnull(docu_meta).sum()

document_id           0
source_id          2518
publisher_id      64024
publish_time    1011118
dtype: int64

In [27]:
# evenets의 NaN 값.
pd.isnull(events).sum()

display_id        0
uuid              0
document_id       0
timestamp         0
platform          0
geo_location    340
dtype: int64

In [25]:
# merge했을 때의 NaN 값.
pd.isnull(merged).sum()

display_id            0
uuid                  0
document_id           0
timestamp             0
platform              0
geo_location        340
source_id          1322
publisher_id       1323
publish_time    3566273
dtype: int64

In [28]:
# 필요한 변수들만 할당
new_merged = merged.loc[:,['document_id','timestamp','publish_time']]

In [29]:
# NaN값 확인
new_merged.isnull().sum()

document_id           0
timestamp             0
publish_time    3566273
dtype: int64

In [30]:
new_merged.shape

(23120126, 3)

In [31]:
# from datetime import datetime
# def convert_odd_timestamp(timestamp_ms_relative):
#     TIMESTAMP_DELTA=1465876799998
#     return datetime.fromtimestamp((int(timestamp_ms_relative)+TIMESTAMP_DELTA)//1000)

In [75]:
# timestamp를 datetime으로 변환시키는 함수 정의
import datetime
def my_date(raw_date):
  added_date = raw_date + 1465876799998
  revised_date = datetime.datetime.fromtimestamp(added_date // 1000)
  return revised_date

In [78]:
new_merged['timestamp'] = new_merged['timestamp'].apply(lambda x: float(x))

In [79]:
my_date(new_merged['timestamp'])

TypeError: cannot convert the series to <class 'float'>

In [38]:
new_merged.head()

,document_id,timestamp,publish_time
0,379743,61,NaN
1,379743,74476960,NaN
2,379743,78297892,NaN
3,379743,148985378,NaN
4,379743,177159039,NaN


In [67]:
new_merged['diff'] = new_merged['publish_time'] - new_merged['timestamp']

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('S21') dtype('S21') dtype('S21')

,display_id,uuid,document_id,timestamp,publish_time
0,1,cb8c55702adb93,379743,61,NaN
1,1239772,a2eb3a4e0a4c08,379743,74476960,NaN
2,1306670,c9d0cf076200f3,379743,78297892,NaN
3,2357895,6e0aabe592ba61,379743,148985378,NaN
4,2785946,1eb4bef36f8623,379743,177159039,NaN


In [70]:
new_merged['new_timestamp'] = new_merged['timestamp'] + 1465876799998 

In [71]:
new_merged.head()

,display_id,uuid,document_id,timestamp,publish_time,new_timestamp
0,1,cb8c55702adb93,379743,61,NaN,1465876800059
1,1239772,a2eb3a4e0a4c08,379743,74476960,NaN,1465951276958
2,1306670,c9d0cf076200f3,379743,78297892,NaN,1465955097890
3,2357895,6e0aabe592ba61,379743,148985378,NaN,1466025785376
4,2785946,1eb4bef36f8623,379743,177159039,NaN,1466053959037


In [202]:
# timestamp를 datetime으로 변환시키는 함수 정의
def my_date(raw_date):
  added_date = raw_date + 1465876799998
  revised_date = pd.to_datetime((added_date // 1000), unit='s', utc=True)
  return revised_date

In [203]:
# timestamp를 datetime으로 변환시켜 새로운 열에 할당
new_merged['new_timestamp'] = my_date(new_merged['timestamp'])

In [204]:
# 내림차순으로 확인
new_merged.sort_values('new_timestamp', ascending=True).head()

,display_id,uuid,document_id,timestamp,publish_time,new_timestamp
0,1,cb8c55702adb93,379743,61,NaN,2016-06-14 04:00:00+00:00
30,2,79a85fa78311b9,1794259,81,2016-06-14 00:00:00,2016-06-14 04:00:00+00:00
319463,4,85281d0a49f7ac,1777797,234,NaN,2016-06-14 04:00:00+00:00
350301,15,24c64dc30891c0,1777136,1000,2016-06-13 16:00:00,2016-06-14 04:00:00+00:00
260,3,822932ce3d8757,1179111,182,NaN,2016-06-14 04:00:00+00:00


In [276]:
new_merged.head()

,display_id,uuid,document_id,timestamp,publish_time,new_timestamp,new_publish_time
0,1,cb8c55702adb93,379743,61,NaN,2016-06-14 04:00:00+00:00,NaT
1,1239772,a2eb3a4e0a4c08,379743,74476960,NaN,2016-06-15 00:41:16+00:00,NaT
2,1306670,c9d0cf076200f3,379743,78297892,NaN,2016-06-15 01:44:57+00:00,NaT
3,2357895,6e0aabe592ba61,379743,148985378,NaN,2016-06-15 21:23:05+00:00,NaT
4,2785946,1eb4bef36f8623,379743,177159039,NaN,2016-06-16 05:12:39+00:00,NaT


In [275]:
# publish_time이 이상한 게 있다. (2870개)
new_merged[new_merged['publish_time'] == '1412-01-24 00:00:00'].shape

(2870, 7)

In [254]:
# new_merged['new_publish_time'] = new_merged['publish_time'].replace(('1412-01-24 00:00:00'), '1970-01-01 00:00:00')
# new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('0002-11-27 19:00:00'), '1970-01-01 00:00:00')
# new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('0017-06-30 19:00:00'), '1970-01-01 00:00:00')
new_merged.ix[new_merged['new_publish_time'] < '1970-01-01 00:00:00'] = '1970-01-01 00:00:00'
new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('3012-05-03 00:00:00'), '2016-06-28 00:00:00')

In [263]:
new_merged['new_publish_time'] = pd.to_datetime(new_merged['new_publish_time'], format='%Y-%m-%d %H:%M:%S')

In [278]:
new_merged['new_timestamp'] = pd.to_datetime(new_merged['new_timestamp'], format='%Y-%m-%d %H:%M:%S', utc=True)

In [272]:
# NaT의 개수
(new_merged['new_publish_time'] == (new_merged['new_publish_time'][0])).sum()

0

In [273]:
new_merged['new_publish_time'].isnull().sum()

3566273

In [279]:
new_merged['diff'] = new_merged['new_publish_time'] - new_merged['new_timestamp']

ValueError: Incompatible tz's on datetime subtraction ops

In [262]:
new_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23120126 entries, 0 to 23120125
Data columns (total 7 columns):
display_id          object
uuid                object
document_id         object
timestamp           object
publish_time        object
new_timestamp       object
new_publish_time    datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 2.0+ GB


In [167]:
type(new_merged['publish_time'][30])

str

In [168]:
type(new_merged['new_timestamp'][30])

pandas._libs.tslib.Timestamp

In [151]:
type(pd.to_datetime(new_merged['publish_time'][30]))

pandas._libs.tslib.Timestamp

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23120126 entries, 0 to 23120125
Data columns (total 6 columns):
display_id       int64
uuid             object
document_id      int64
timestamp        int64
publish_time     object
new_timestamp    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(3), object(2)
memory usage: 1.8+ GB


In [170]:
import datetime
 
one = datetime.datetime.strptime('2015-04-15 00:00:00', '%Y-%m-%d %H:%M:%S')

In [173]:
pd.to_datetime(one)

Timestamp('2015-04-15 00:00:00')